## Configurações Iniciais

In [1]:
#Bibliotecas Utilizadas
import IPython as IP
import CoolProp
import CoolProp.CoolProp as CP #Propriedades termodinâmicas
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import psychrolib as psy # Check docmentation at: https://psychrometrics.github.io/psychrolib/api_docs.html
from itertools import product
from scipy.interpolate import interp1d # Biblio para Interpolação
from tqdm import tqdm

# Configuração Inicial da biblio de psicometria
psy.SetUnitSystem(psy.SI)

## Moiller Diagram for R410A

![](r410a-moiller.png)

## Dados do Compressor

In [2]:
compressor_data = pd.read_csv('fugitsu_compressor_data.csv')

In [3]:
# Colunas da tabela
compressor_data.columns

Index(['Rot', 'Condicao', 'T_evap', 'T_cond', 'T_sup', 'T_sub', 'CAP', 'W',
       'V_cil', 'P_evap', 'P_cond', 'RP', 'h_1', 'h_3', 'm', 'v_1', 'eta_v',
       's_1', 'h_2s', 'W_s', 'eta_g', 'Sigma'],
      dtype='object')

In [4]:
# Amostra dos dados do compressor
compressor_data.head()

,Rot,Condicao,T_evap,T_cond,T_sup,T_sub,CAP,W,V_cil,P_evap,...,h_1,h_3,m,v_1,eta_v,s_1,h_2s,W_s,eta_g,Sigma
0,4800,C,15.0,45.0,20.0,35.0,9572.17,1691.91,0.000015,1.254290e+06,...,431305.922776,256662.500668,0.054810,0.021332,0.974356,1803.36268,452557.513419,1164.795307,0.688450,1.000000
1,3000,C,15.0,45.0,20.0,35.0,5705.38,944.62,0.000015,1.254290e+06,...,431305.922776,256662.500668,0.032669,0.021332,0.929205,1803.36268,452557.513419,694.262622,0.734965,0.625000
2,4000,C,15.0,45.0,20.0,35.0,7876.02,1326.42,0.000015,1.254290e+06,...,431305.922776,256662.500668,0.045098,0.021332,0.962044,1803.36268,452557.513419,958.398266,0.722545,0.833333
3,4800,I,15.0,50.0,20.0,35.0,9565.05,1902.83,0.000015,1.254290e+06,...,431305.922776,256386.384387,0.054683,0.021332,0.972094,1803.36268,455810.126532,1339.952851,0.704189,1.000000
4,4000,I,15.0,50.0,20.0,35.0,7861.63,1507.11,0.000015,1.254290e+06,...,431305.922776,256386.384387,0.044944,0.021332,0.958771,1803.36268,455810.126532,1101.323415,0.730752,0.833333


In [5]:
# Range de RP para cada uma das rotações
print('Range RP para Rotação de 3000. Mínimo: ', 
      compressor_data[compressor_data[ 'Rot'] == 3000]['RP'].min(), 
      ' Máximo: ', compressor_data[compressor_data[ 'Rot'] == 3000]['RP'].max())

print('Range RP para Rotação de 4000. Mínimo: ', 
      compressor_data[compressor_data[ 'Rot'] == 4000]['RP'].min(), 
      ' Máximo: ', compressor_data[compressor_data[ 'Rot'] == 4000]['RP'].max())

print('Range RP para Rotação de 4800. Mínimo: ', 
      compressor_data[compressor_data[ 'Rot'] == 4800]['RP'].min(), 
      ' Máximo: ', compressor_data[compressor_data[ 'Rot'] == 4800]['RP'].max())

Range RP para Rotação de 3000. Mínimo:  2.173446345670816  Máximo:  4.1094359555634234
Range RP para Rotação de 4000. Mínimo:  2.173446345670816  Máximo:  4.1094359555634234
Range RP para Rotação de 4800. Mínimo:  2.173446345670816  Máximo:  4.1094359555634234


In [6]:
# Interpolações para calcular eta_v e eta_g a partir de RP
def eta_from_rp_calc(RP,rot,kind): 
    if kind == 'v' and rot in [3000/60,4000/60,4800/60]:
            interp_cubica = interp1d(compressor_data[compressor_data['Rot']==rot*60]['RP'], compressor_data[compressor_data['Rot']==rot*60]['eta_v'], kind='cubic')
    elif kind == 'g' and rot in [3000/60,4000/60,4800/60]:
            interp_cubica = interp1d(compressor_data[compressor_data['Rot']==rot*60]['RP'], compressor_data[compressor_data['Rot']==rot*60]['eta_g'], kind='cubic')
    else:
        print("""Insira um valor válido de Rotação do compressor [ 3000,4000,4800] e escolha 
        'v' ou 'g' para escolher qual eficiência calcular""")
        print('RP: ',RP,' rot: ',rot,' kind: ',kind)
        return
    
    if RP > 2.174 and RP < 4.1094:
        return float(interp_cubica(RP))
    elif RP <= 2.174:
        return float(interp_cubica(2.174))
    elif RP >= 4.1094:
        return float(interp_cubica(4.1094))

In [7]:
# Testando a função
eta_from_rp_calc(2.3,3000/60,'v')

0.9217048435199436

### Plotando a Variacao da Eficiencia Vol. e Global. com RP

In [ ]:
plot

## Etapa de Calibração - Encontrar UAs Iniciais a partir dos dados da Whirlpool

### Variáveis Globais - Dados AC Split Inveter da Fujitsu

In [8]:
# Constantes de Mudança de Unidade
CONST_FROM_BTU_TO_SI = 0.293
CONST_FROM_PSI_TO_SI = 6894.76
ATM_PRESSURE = 101325 # SI (N/mˆ2)
CONST_FROM_CELSIUS_TO_K = 273.15

# Líquido Refrigerante utilizando
coolant = 'R410A'

# Parâmetros de fabricação do AC
ROT = [3000/60,4000/60,4800/60] # Vetor com as possíveis rotações do compressor testadas em Hz
VOL_cil = 15e-6 # Volume do cilindo do compressor em mˆ3 
delta_sub = 4.29 # Delta de sub-resfriamento
delta_sup = 3.02 # Delta de super-aquecimento
T_amb = 34.96 + CONST_FROM_CELSIUS_TO_K # Temperatura do ambiente externo (K)
T_int_tbs = 26.7 + CONST_FROM_CELSIUS_TO_K # Temperatura de Bulbo seco do ambiente interno (K)
T_int_tbu = 19.4 + CONST_FROM_CELSIUS_TO_K # Temperatura de Bulbo úmido do ambiente interno (K)

## Funções auxiliares a biblioteca CoolProp

In [9]:
# Calc H a partir de T e Q
def GetEnthalpy_fromTandQ(T,Q,liquid):
    if T > 273.15 and T < 647.096:
        result = CP.PropsSI('H','T',T,'Q',Q,liquid)
    elif T <= 273.08:
        result = CP.PropsSI('H','T',273.15,'Q',Q,liquid)
    elif T >= 647.096:
        result = CP.PropsSI('H','T',647.096,'Q',Q,liquid)
    return result

# Calc P a partir de T e Q
def GetPressure_fromTandQ(T,Q,liquid):
    if T > 273.15 and T < 647.096:
        result = CP.PropsSI('P','T',T,'Q',Q,liquid)
    elif T <= 273.08:
        result = CP.PropsSI('P','T',273.15,'Q',Q,liquid)
    elif T >= 647.096:
        result = CP.PropsSI('P','T',647.096,'Q',Q,liquid)
    return result

# Calc H a partir de T e P
def GetEnthalpy_fromTandP(T,P,liquid):
    if T > 273.15 and T < 647.096:
        result = CP.PropsSI('H','T',T,'P',P,liquid)
    elif T <= 273.08:
        result = CP.PropsSI('H','T',273.15,'P',P,liquid)
    elif T >= 647.096:
        result = CP.PropsSI('H','T',647.096,'P',P,liquid)
    return result

# Calc D a partir de T e P
def GetDensity_fromTandP(T,P,liquid):
    if T > 273.15 and T < 647.096:
        result = CP.PropsSI('D','T',T,'P',P,liquid)
    elif T <= 273.08:
        result = CP.PropsSI('D','T',273.15,'P',P,liquid)
    elif T >= 647.096:
        result = CP.PropsSI('D','T',647.096,'P',P,liquid)
    return result

# Calc S a partir de T e P
def GetEntropy_fromTandP(T,P,liquid):
    if T > 273.15 and T < 647.096:
        result = CP.PropsSI('S','T',T,'P',P,liquid)
    elif T <= 273.08:
        result = CP.PropsSI('S','T',273.15,'P',P,liquid)
    elif T >= 647.096:
        result = CP.PropsSI('S','T',647.096,'P',P,liquid)
    return result

### Função para calibração

In [10]:
# Função para cálculo de UAs partindo das variáveis: T_2, T_cond, T_evap e Q_evap_lat
def CalibrateUAs (T_2,T_cond,T_evap,Q_dot_evap_lat, rot_comp=ROT[2]):
    # Pressões no Evaporador e no Condensador
    P_evap = GetPressure_fromTandQ(T_evap,1,coolant) # (Pa)
    P_cond = GetPressure_fromTandQ(T_cond,0,coolant) # (Pa)
    
    # Encontrando as eficiências volumétricas e globais, a partir dos dados do compressor
    PR = P_cond/P_evap # (-)
        
    # Encontrando os valores de Eficiência Gloval e Volumétrica a partir dos dados 
    eta_v = eta_from_rp_calc(PR,rot_comp,'v') # (-)
    eta_g = eta_from_rp_calc(PR,rot_comp,'g') # (-)
    
    # Ponto 1 (Entrada do Compressor)
    T_1 = T_evap + delta_sup # Temperatura no ponto 1 (K)
    v_1 = 1/GetDensity_fromTandP(T_1,P_evap,coolant) # Densidade Volumétrica do ponto 1 (mˆ3/Kg)
    h_1 = GetEnthalpy_fromTandP(T_1,P_evap,coolant) # Entalpia real no ponto 1 (J/kg)
    s_1 = GetEntropy_fromTandP(T_1,P_evap,coolant) # Entropia do ponto 1 (J/kg)
    
    # Fluxo do refrigerante
    m_dot = eta_v*rot_comp*VOL_cil/v_1 # Vazão mássica (Kg/s)
    
    # Ponto 2 (Saída do Compressor - Entrada no condensador)
    h_2s = CP.PropsSI('H','P',P_cond,'S',s_1,coolant) # Entalpia Isentrópica do ponto 2 (J/kg)
    Pot = m_dot*(h_2s-h_1)/eta_g # Potência consumida pelo compressor (W)
    h_2a = h_1 + Pot/m_dot # Entalpia adiabática do ponto 2 (J/kg)
    T_2a = CP.PropsSI('T','P',P_cond,'H',h_2a,coolant) # Temperatura no ponto 2a (K)
    
    h_2 = GetEnthalpy_fromTandP(T_2,P_cond,coolant) # Entalpia real do ponto 2 (J/kg)
    Q_dot_comp = abs(Pot - m_dot*(h_2 - h_1)) # Calor perdido para o ambiente no compressor - BE no refrigerante
    UA_comp = Q_dot_comp/(T_2a-T_amb) # Coef. global de Transferencia de calor X Área do Compressor - BE no exterior
    
    # Ponto 3 (Saída do Condensador)
    T_3 = T_cond - delta_sub # Temperatura no ponto 3
    h_3 = GetEnthalpy_fromTandP(T_3,P_cond,coolant) # Entalpia real no ponto 3
    Q_dot_cond = m_dot*(h_2-h_3) # Calor perdido para o ambiente no condensador - BE no refrigerante
    UA_cond = Q_dot_cond/(T_cond-T_amb) # Coef. global de Transferencia de calor X Área do Condensador - BE no exterior
    
    # No evaporador
    Q_dot_evap_sens = m_dot*(h_1-h_3) - Q_dot_evap_lat
    UA_evap_sens = Q_dot_evap_sens/(T_int_tbs - T_evap)
    
    
    # Cálculo Psicométricos do Ar no ambiente interior (onde fica instalado o evaporador)
    psy.SetUnitSystem(psy.SI)
    W_interior = psy.GetHumRatioFromTWetBulb(T_int_tbs - CONST_FROM_CELSIUS_TO_K,T_int_tbu - CONST_FROM_CELSIUS_TO_K,ATM_PRESSURE)
    W_evap = psy.GetHumRatioFromRelHum(T_evap - CONST_FROM_CELSIUS_TO_K,1,P_evap)
    
    # Entalpia de Vaporização (Diferença entre a entalpia do valor e o líquido saturado a T_evap)
    hlv = GetEnthalpy_fromTandQ(T_evap,1,'Water') - GetEnthalpy_fromTandQ(T_evap,0,'Water')  # (J/kg)
    
    # No balanço de energia latente no evaporador
    UA_evap_lat = Q_dot_evap_lat/((W_interior - W_evap)*hlv)
    
    # Coeficiente de Rendimento
    COP = (Q_dot_evap_sens + Q_dot_evap_lat)/Pot
    
    # Armazenando as variáveis calibradas em um dicionário
    variaveis_calibradas = {}
    
    # Eficiências
    variaveis_calibradas['eta_v'] = eta_v
    variaveis_calibradas['eta_g'] = eta_g
    
    # Temperaturas
    variaveis_calibradas['T_1'] = T_1
    variaveis_calibradas['T_2'] = T_2
    variaveis_calibradas['T_2a'] = T_2a
    variaveis_calibradas['T_3'] = T_3
    variaveis_calibradas['T_evap'] = T_evap
    variaveis_calibradas['T_cond'] = T_cond
    
    # Entalpias
    variaveis_calibradas['h_1'] = h_1
    variaveis_calibradas['h_2'] = h_2
    variaveis_calibradas['h_2a'] = h_2a
    variaveis_calibradas['h_2s'] = h_2s
    variaveis_calibradas['h_3'] = h_3
    variaveis_calibradas['hlv'] = hlv
    
    # Pressões
    variaveis_calibradas['P_evap'] = P_evap
    variaveis_calibradas['P_cond'] = P_cond

    # Volumes Específicos e Entopias
    variaveis_calibradas['v_1'] = v_1
    variaveis_calibradas['s_1'] = s_1
    
    # Vazão mássica
    variaveis_calibradas['m_dot'] = m_dot
    
    # Trocas de calor
    variaveis_calibradas['Pot'] = Pot
    variaveis_calibradas['Q_dot_comp'] = Q_dot_comp
    variaveis_calibradas['Q_dot_cond'] = Q_dot_cond
    variaveis_calibradas['Q_dot_evap_sens'] = Q_dot_evap_sens
    variaveis_calibradas['Q_dot_evap_lat'] = Q_dot_evap_lat
    
    # Resistências Térmicas
    variaveis_calibradas['UA_comp'] = UA_comp 
    variaveis_calibradas['UA_cond'] = UA_cond 
    variaveis_calibradas['UA_evap_sens'] = UA_evap_sens 
    variaveis_calibradas['UA_evap_lat'] = UA_evap_lat
    
    # Umidades absolutas
    variaveis_calibradas['W_interior'] = W_interior
    variaveis_calibradas['W_evap'] = W_evap
    
    # Coeficiente de Rendimento
    variaveis_calibradas['COP'] = COP

    return variaveis_calibradas

# Calibragem

In [11]:
# Parâmetros auxiliares para calibragem
T_2 = 77.76 + CONST_FROM_CELSIUS_TO_K # Temperatura de descarga do compressor em K
T_evap = 10.86 + CONST_FROM_CELSIUS_TO_K # Temperatura de saída no evaporador em K
T_cond = 45.38 + CONST_FROM_CELSIUS_TO_K # Temperatura no meio do condensador em K 
Q_dot_evap_lat = 6494.32*CONST_FROM_BTU_TO_SI # Calor latente no evaporador em W

In [12]:
variaveis_calibradas = CalibrateUAs(T_2,T_cond,T_evap,Q_dot_evap_lat)

In [13]:
print('Número de variáveis: ',len(variaveis_calibradas))
print('')
variaveis_calibradas

Número de variáveis:  31



{'eta_v': 0.9688481516931082,
 'eta_g': 0.703186354317819,
 'T_1': 287.03,
 'T_2': 350.90999999999997,
 'T_2a': 343.7721444704031,
 'T_3': 314.23999999999995,
 'T_evap': 284.01,
 'T_cond': 318.53,
 'h_1': 427913.598729894,
 'h_2': 472228.11645543977,
 'h_2a': 463091.1891295062,
 'h_2s': 452650.0002766828,
 'h_3': 267989.1488007014,
 'hlv': 2475150.5110408855,
 'P_evap': 1112661.4884296856,
 'P_cond': 2758341.4406268587,
 'v_1': 0.023801893122229297,
 's_1': 1802.6639715891552,
 'm_dot': 0.04884560131672579,
 'Pot': 1718.270555942538,
 'Q_dot_comp': 446.29870942245,
 'Q_dot_cond': 9976.175187403003,
 'Q_dot_evap_sens': 5908.770162038015,
 'Q_dot_evap_lat': 1902.8357599999997,
 'UA_comp': 12.51463466513754,
 'UA_cond': 957.4064479273501,
 'UA_evap_sens': 373.0284193205823,
 'UA_evap_lat': 0.0741631896375437,
 'W_interior': 0.011093835138280729,
 'W_evap': 0.0007278330577198207,
 'COP': 4.546202514512068}

## Implementação do método de Newton Raphson

In [18]:
def Main_COPCalculator(UA_comp=variaveis_calibradas['UA_comp'],UA_cond=variaveis_calibradas['UA_cond'],
                  UA_evap_sens=variaveis_calibradas['UA_evap_sens'],UA_evap_lat=variaveis_calibradas['UA_evap_lat'],
                  rot_comp=ROT[0]):
    
    """ Função para resolver um ciclo termodinâmico representativo de um sistema de refrigeração com 
    N equações e N incógnitas acopladas. Algoritmo baseado no método de Newton Raphson com 
    multi-variáveis. Após a resolucão do problema, é calculado o COP do ciclo para as variáveis de 
    entrada escolhidas.
    
    Variáveis Globais utilizadas:
        T_amb (Temperatura Ambiente em kelvin)
        T_int_tbs (Temperatura de Bulbo Seco para o ambiente Interior em kelvin)
        T_int_tbu (Temperatura de Bulbo Úmido para o ambiente Interior em kelvin)
        coolant (Líquido Refrigerante utilizado)
        VOL_cil (Volume do cilindro do compressor em metros cúbicos)
        delta_sup (Delta de super-aquecimento)
        CONST_FROM_K_TO_C (Constante responsável por mudar a temperatura de Celcius para Kelvin)
        
    Agrs: 
        UA_comp (float): Resistência térmica do Compressor. Valor calibrado como default
        UA_cond (float): Resistência térmica do Condensador. Valor calibrado como default
        UA_evap_sens (float): Resistênca térmica sensível do Evaporador. Valor calibrado como default
        UA_evap_lat (float): Resistência térmica latente do Evaporador. Valor calibrado como default
        rot_comp (float) : Rotação do compressor
        
    Returns:
        float: Valor do COP do ciclo.
    
    """
    
    # Declaração do vetor com as variáveis - Usando valores calibrados
    X_0 = [variaveis_calibradas['eta_v'], # Eficiência Volumétrica do Compressor -- 0
         variaveis_calibradas['eta_g'], # Eficiência Global do Compressor -- 1
         variaveis_calibradas['T_1'], # Temperatura no ponto de sucção (K) -- 2
         variaveis_calibradas['T_2'], # Temperatura real do ponto de descarga (J/Kg) -- 3
         variaveis_calibradas['T_2a'], # Temperatura Adiabática do ponto de descarga (J/kg) -- 4
         variaveis_calibradas['T_3'], # Temperatura real na saída do condensador (K) -- 5
         variaveis_calibradas['T_evap'], # Temperatura no Evaporador - Saturação (K) -- 6
         variaveis_calibradas['T_cond'], # Temperatura no Condensador - Saturação (K) -- 7 
         variaveis_calibradas['h_1'], # Entalpia real no ponto de sucção (J/kg) -- 8
         variaveis_calibradas['h_2'], # Entalpia real do ponto de descarga (J/kg) -- 9
         variaveis_calibradas['h_2a'], # Entalpia Adiabática do ponto de descarga (J/kg) -- 10
         variaveis_calibradas['h_2s'], # Entalpia Isentrópica do ponto de descarga (J/kg) -- 11
         variaveis_calibradas['h_3'], # Entalpia real na saída do condensador (J/kg) -- 12
         variaveis_calibradas['hlv'], # Entalpia de vaporização da água na temperatura do evaporador -- 13
         variaveis_calibradas['P_evap'], # Pressão do Evaporador (Pa) -- 14
         variaveis_calibradas['P_cond'], # Pressão do Condensador (Pa) -- 15
         variaveis_calibradas['v_1'], # Densidade Volumétrica do ponto de sucção (mˆ3/Kg) -- 16
         variaveis_calibradas['s_1'], # Entropia do ponto de succão (J/kg) -- 17
         variaveis_calibradas['m_dot'], # Vazão mássica (kg/s) -- 18
         variaveis_calibradas['Pot'], # Potência elétrica consumida pelo compressor -- 19
         variaveis_calibradas['Q_dot_comp'], # Calor trocado pelo compressor -- 20
         variaveis_calibradas['Q_dot_cond'], # Calor trocado pelo condensador -- 21
         variaveis_calibradas['Q_dot_evap_sens'], # Calor trocado pelo evaporador de forma sensível -- 22
         variaveis_calibradas['Q_dot_evap_lat'], # Calor trocado pelo evaporador de forma latente -- 23
         variaveis_calibradas['W_interior'], # Umidade real do ambiente interior (Kg agua/Kg ar) -- 24
         variaveis_calibradas['W_evap'], # Umidade real do ar próximo ao evaporador (Kg agua/Kg ar) -- 25
         variaveis_calibradas['COP']] # Coeficiente de performance do sistema -- 26
    
    # Função que modela a relações entre as variáveis do ciclo termodinâmico
    def System(X):
        return [X[14] - GetPressure_fromTandQ(X[6],1,coolant), # Eq (0) 
                X[15] - GetPressure_fromTandQ(X[7],0,coolant), # Eq (1) 
                X[0] - eta_from_rp_calc(X[15]/X[14],rot_comp,'v'), # Eq (2) 
                X[1] - eta_from_rp_calc(X[15]/X[14],rot_comp,'g'), # Eq (3) 
                X[2] - X[6] - delta_sup, # Eq (4) 
                X[16] - 1/GetDensity_fromTandP(X[2],X[14],coolant), # Eq (5) 
                X[8] - GetEnthalpy_fromTandP(X[2],X[14],coolant), # Eq (6) 
                X[17] - GetEntropy_fromTandP(X[2],X[14],coolant), # Eq (7) 
                X[18] - X[0]*rot_comp*VOL_cil/X[16], # Eq (8) 
                X[11] - CP.PropsSI('H','P',X[15],'S',X[17],coolant), # Eq (9) 
                X[19] - (X[18]*(X[11] - X[8]))/X[1], # Eq (10) 
                X[10] - X[8] - X[19]/X[18], # Eq (11) 
                X[4] - CP.PropsSI('T','P',X[15],'H',X[10],coolant), # Eq (12) 
                X[20] - UA_comp*(X[4] - T_amb), # Eq (13) 
                X[9] - X[8] - ((abs(X[19] - X[20]))/X[18]), # Eq (14) 
                X[3] - CP.PropsSI('T','P',X[15],'H',X[9],coolant), # Eq (15) 
                X[5] - X[7] + delta_sub, # Eq (16) 
                X[12] - GetEnthalpy_fromTandP(X[5],X[15],coolant), # Eq (17) 
                X[21] - X[18]*(X[9] - X[12]), # Eq (18) 
                X[7] - T_amb - X[21]/UA_cond, # Eq (19) 
                X[24] - psy.GetHumRatioFromTWetBulb(T_int_tbs - CONST_FROM_CELSIUS_TO_K,T_int_tbu - CONST_FROM_CELSIUS_TO_K,ATM_PRESSURE), # Eq (20) 
                X[25] - psy.GetHumRatioFromRelHum(X[6] - CONST_FROM_CELSIUS_TO_K,1,X[14]), # Eq (21) 
                X[13] - GetEnthalpy_fromTandQ(X[6],1,'Water') + GetEnthalpy_fromTandQ(X[6],0,'Water'), # Eq (22) 
                X[22] - UA_evap_sens*(T_int_tbs - X[6]), # Eq (23) 
                X[23] - UA_evap_lat*(X[24] - X[25])*X[13], # Eq (24) 
                X[22] + X[23] - X[18]*(X[8] + X[12]), # Eq (25)
                X[26] - (X[22] + X[23])/X[19]],[[14,6],[15,7],[0,15,14],
                                                [1,15,14],[2,6],[16,2,14],
                                                 [8,2,14],[17,2,14],[18,0,16],
                                                [11,15,17],[19,18,11,8,1],[10,8,19,18],
                                                 [4,15,10],[20,4],[9,8,19,20,18],
                                                 [3,15,9],[5,7],[12,5,15],
                                                 [21,18,9,12],[7,21],[24],
                                                 [25,6,14],[13,6],[22,6],
                                                 [23,24,25,13],[22,23,18,8,12],[26,22,23,19]] 
    
    # Jacobiana das funções que modelam o sistema
    def Jacob(F_var,X):
        J = []
        for i in range(len(X)):
            new_line = []
            for j in range(len(X)):
                if j in F_var[i]:
                    X_inc = X.copy()
                    X_inc[j] += 0.01
                    F_inc = System(X_inc)[0]
                    F_org = System(X)[0]
                    new_line.append((F_inc[i] - F_org[i])/0.01) 
                else:
                    new_line.append(0)
            J.append(new_line)
        return J
    
    # Algoritmo de Newton Raphson
    def NewtonRaphson(X,eps=1):
        X = X.copy()
        F_value,F_value_var = System(X)
        F_value = np.array(F_value,dtype='float')
        F_norm = np.linalg.norm(F_value, ord=2)
        ite_counter = 0
        pbar = tqdm(total=100)
        while abs(F_norm) > eps and ite_counter < 100:
            delta = np.linalg.solve(np.array(Jacob(F_value_var,X),dtype='float'), -F_value)
            X = X + delta/5
            F_value,F_value_var = System(X)
            F_value = np.array(F_value,dtype='float')
            F_norm = np.linalg.norm(F_value, ord=2)
            ite_counter += 1
            pbar.update(1)
        pbar.close()
        
        final_norm = abs(F_norm)
        if final_norm > eps:
            ite_counter = -1
        return X,ite_counter,final_norm
            
    # Implementando o algoritmo de Newton Raphson
    solved, ite_counter,final_norm = NewtonRaphson(X_0)
    return solved,final_norm,System(solved)


In [19]:
Main_COPCalculator()

 55%|█████▌    | 55/100 [00:53<00:43,  1.04it/s]


(array([8.94743964e-01, 7.19215107e-01, 2.70952194e+02, 3.24144803e+02,
        3.38064866e+02, 3.07845740e+02, 2.67932194e+02, 3.12135740e+02,
        4.21387897e+05, 4.44280475e+05, 4.62551108e+05, 4.50993605e+05,
        2.56426828e+05, 2.50093846e+06, 7.98082639e+05, 2.36651848e+06,
        3.27068996e-02, 1.81057789e+03, 2.05174078e-02, 8.44577194e+02,
        3.74874202e+02, 3.85426939e+03, 1.19062486e+04, 2.00062951e+03,
        1.10938351e-02, 3.07466347e-04, 1.64660696e+01]),
 0.8718905654397755,
 ([-0.34068050037603825,
   -0.028839778155088425,
   2.4329733250283425e-07,
   -2.2525417620045118e-07,
   3.863576125695545e-14,
   -9.08622372480794e-09,
   -0.020826937805395573,
   -8.350547864210967e-05,
   8.638837744809336e-08,
   0.00012549239909276366,
   0.0004115263917583434,
   -0.7224092176984414,
   -5.454585902953113e-07,
   -4.547473508864641e-13,
   -0.32427397161882254,
   -1.196595917463128e-06,
   -2.042810365310288e-14,
   -0.00036119797732681036,
   -0.00048059

In [31]:
CP.PropsSI("D","T",273.09,"P",796303.2393,"R410A")

30.503359607011806

## Calibragem - Segundo Método

In [ ]:
def ModelCalibration (T_2,T_evap,T_cond,Q_dot_evap_lat):
    """ Função para resolver um ciclo termodinâmico representativo de um sistema de refrigeração com 
    N equações e N incógnitas acopladas. Algoritmo baseado no método de Newton Raphson com 
    multi-variáveis. Após a resolucão do problema, é calculado o COP do ciclo para as variáveis de 
    entrada escolhidas.
    
    Variáveis Globais utilizadas:
        T_amb (Temperatura Ambiente em kelvin)
        T_int_tbs (Temperatura de Bulbo Seco para o ambiente Interior em kelvin)
        T_int_tbu (Temperatura de Bulbo Úmido para o ambiente Interior em kelvin)
        coolant (Líquido Refrigerante utilizado)
        VOL_cil (Volume do cilindro do compressor em metros cúbicos)
        delta_sup (Delta de super-aquecimento)
        CONST_FROM_K_TO_C (Constante responsável por mudar a temperatura de Celcius para Kelvin)
        
    Agrs: 
        UA_comp (float): Resistência térmica do Compressor. Valor calibrado como default
        UA_cond (float): Resistência térmica do Condensador. Valor calibrado como default
        UA_evap_sens (float): Resistênca térmica sensível do Evaporador. Valor calibrado como default
        UA_evap_lat (float): Resistência térmica latente do Evaporador. Valor calibrado como default
        rot_comp (float) : Rotação do compressor
        
    Returns:
        float: Valor do COP do ciclo.
    
    """
    
    # Declaração do vetor com as variáveis - Usando valores calibrados
    X_0 = [variaveis_calibradas['eta_v'], # Eficiência Volumétrica do Compressor -- 0
         variaveis_calibradas['eta_g'], # Eficiência Global do Compressor -- 1
         variaveis_calibradas['T_1'], # Temperatura no ponto de sucção (K) -- 2
#          variaveis_calibradas['T_2'], # Temperatura real do ponto de descarga (J/Kg) -- 3
         variaveis_calibradas['T_2a'], # Temperatura Adiabática do ponto de descarga (J/kg) -- 4
         variaveis_calibradas['T_3'], # Temperatura real na saída do condensador (K) -- 5
#          variaveis_calibradas['T_evap'], # Temperatura no Evaporador - Saturação (K) -- 6
#          variaveis_calibradas['T_cond'], # Temperatura no Condensador - Saturação (K) -- 7 
         variaveis_calibradas['h_1'], # Entalpia real no ponto de sucção (J/kg) -- 8
         variaveis_calibradas['h_2'], # Entalpia real do ponto de descarga (J/kg) -- 9
         variaveis_calibradas['h_2a'], # Entalpia Adiabática do ponto de descarga (J/kg) -- 10
         variaveis_calibradas['h_2s'], # Entalpia Isentrópica do ponto de descarga (J/kg) -- 11
         variaveis_calibradas['h_3'], # Entalpia real na saída do condensador (J/kg) -- 12
         variaveis_calibradas['hlv'], # Entalpia de vaporização da água na temperatura do evaporador -- 13
         variaveis_calibradas['P_evap'], # Pressão do Evaporador (Pa) -- 14
         variaveis_calibradas['P_cond'], # Pressão do Condensador (Pa) -- 15
         variaveis_calibradas['v_1'], # Densidade Volumétrica do ponto de sucção (mˆ3/Kg) -- 16
         variaveis_calibradas['s_1'], # Entropia do ponto de succão (J/kg) -- 17
         variaveis_calibradas['m_dot'], # Vazão mássica (kg/s) -- 18
         variaveis_calibradas['Pot'], # Potência elétrica consumida pelo compressor -- 19
         variaveis_calibradas['Q_dot_comp'], # Calor trocado pelo compressor -- 20
         variaveis_calibradas['Q_dot_cond'], # Calor trocado pelo condensador -- 21
         variaveis_calibradas['Q_dot_evap_sens'], # Calor trocado pelo evaporador de forma sensível -- 22
#          variaveis_calibradas['Q_dot_evap_lat'], # Calor trocado pelo evaporador de forma latente -- 23
         variaveis_calibradas['W_interior'], # Umidade real do ambiente interior (Kg agua/Kg ar) -- 24
         variaveis_calibradas['W_evap'], # Umidade real do ar próximo ao evaporador (Kg agua/Kg ar) -- 25
         variaveis_calibradas['COP']] # Coeficiente de performance do sistema -- 26
    
    # Função que modela a relações entre as variáveis do ciclo termodinâmico
    def System(X):
        return [X[14] - GetPressure_fromTandQ(X[6],1,coolant), # Eq (0) 
                X[15] - GetPressure_fromTandQ(X[7],0,coolant), # Eq (1) 
                X[0] - eta_from_rp_calc(X[15]/X[14],rot_comp,'v'), # Eq (2) 
                X[1] - eta_from_rp_calc(X[15]/X[14],rot_comp,'g'), # Eq (3) 
                X[2] - X[6] - delta_sup, # Eq (4) 
                X[16] - 1/GetDensity_fromTandP(X[2],X[14],coolant), # Eq (5) 
                X[8] - GetEnthalpy_fromTandP(X[2],X[14],coolant), # Eq (6) 
                X[17] - GetEntropy_fromTandP(X[2],X[14],coolant), # Eq (7) 
                X[18] - X[0]*rot_comp*VOL_cil/X[16], # Eq (8) 
                X[11] - CP.PropsSI('H','P',X[15],'S',X[17],coolant), # Eq (9) 
                X[19] - (X[18]*(X[11] - X[8]))/X[1], # Eq (10) 
                X[10] - X[8] - X[19]/X[18], # Eq (11) 
                X[4] - CP.PropsSI('T','P',X[15],'H',X[10],coolant), # Eq (12) 
                X[20] - UA_comp*(X[4] - T_amb), # Eq (13) 
                X[9] - X[8] - ((abs(X[19] - X[20]))/X[18]), # Eq (14) 
                X[3] - CP.PropsSI('T','P',X[15],'H',X[9],coolant), # Eq (15) 
                X[5] - X[7] + delta_sub, # Eq (16) 
                X[12] - GetEnthalpy_fromTandP(X[5],X[15],coolant), # Eq (17) 
                X[21] - X[18]*(X[9] - X[12]), # Eq (18) 
                X[7] - T_amb - X[21]/UA_cond, # Eq (19) 
                X[24] - psy.GetHumRatioFromTWetBulb(T_int_tbs - CONST_FROM_CELSIUS_TO_K,T_int_tbu - CONST_FROM_CELSIUS_TO_K,ATM_PRESSURE), # Eq (20) 
                X[25] - psy.GetHumRatioFromRelHum(X[6] - CONST_FROM_CELSIUS_TO_K,1,X[14]), # Eq (21) 
                X[13] - GetEnthalpy_fromTandQ(X[6],1,'Water') + GetEnthalpy_fromTandQ(X[6],0,'Water'), # Eq (22) 
                X[22] - UA_evap_sens*(T_int_tbs - X[6]), # Eq (23) 
                X[23] - UA_evap_lat*(X[24] - X[25])*X[13], # Eq (24) 
                X[22] + X[23] - X[18]*(X[8] + X[12]), # Eq (25)
                X[26] - (X[22] + X[23])/X[19]],[[14,6],[15,7],[0,15,14],
                                                [1,15,14],[2,6],[16,2,14],
                                                 [8,2,14],[17,2,14],[18,0,16],
                                                [11,15,17],[19,18,11,8,1],[10,8,19,18],
                                                 [4,15,10],[20,4],[9,8,19,20,18],
                                                 [3,15,9],[5,7],[12,5,15],
                                                 [21,18,9,12],[7,21],[24],
                                                 [25,6,14],[13,6],[22,6],
                                                 [23,24,25,13],[22,23,18,8,12],[26,22,23,19]] 
    
    # Jacobiana das funções que modelam o sistema
    def Jacob(F_var,X):
        J = []
        for i in range(len(X)):
            new_line = []
            for j in range(len(X)):
                if j in F_var[i]:
                    X_inc = X.copy()
                    X_inc[j] += 0.01
                    F_inc = System(X_inc)[0]
                    F_org = System(X)[0]
                    new_line.append((F_inc[i] - F_org[i])/0.01) 
                else:
                    new_line.append(0)
            J.append(new_line)
        return J
    
    # Algoritmo de Newton Raphson
    def NewtonRaphson(X,eps=1):
        X = X.copy()
        F_value,F_value_var = System(X)
        F_value = np.array(F_value,dtype='float')
        F_norm = np.linalg.norm(F_value, ord=2)
        ite_counter = 0
        pbar = tqdm(total=100)
        while abs(F_norm) > eps and ite_counter < 100:
            delta = np.linalg.solve(np.array(Jacob(F_value_var,X),dtype='float'), -F_value)
            X = X + delta/5
            F_value,F_value_var = System(X)
            F_value = np.array(F_value,dtype='float')
            F_norm = np.linalg.norm(F_value, ord=2)
            ite_counter += 1
            pbar.update(1)
        pbar.close()
        
        final_norm = abs(F_norm)
        if final_norm > eps:
            ite_counter = -1
        return X,ite_counter,final_norm
            
    # Implementando o algoritmo de Newton Raphson
    solved, ite_counter,final_norm = NewtonRaphson(X_0)
    return solved,final_norm,System(solved)